Here  I have tried with LSTM and CNN using pytorch

TODO: Ensembling both model results. 

Basic data modeling used from this kernel
https://www.kaggle.com/yaroslavmavliutov/riiid-prediction-cnn-keras-0-751

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm

import riiideducation

from sklearn.metrics import roc_auc_score,accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import torch.nn as nn
import torch 
from torch.utils.data import Dataset, DataLoader

In [ ]:
lectures_csv = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/lectures.csv")
example_test_csv = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/example_test.csv")
train_csv = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/train.csv", low_memory=False, nrows=1000000)
questions_csv = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/questions.csv")

In [ ]:
# 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture. So, let's keep just the questions
train_csv = train_csv[train_csv.content_type_id == 0]
# read -1 as null, for lectures
train_csv = train_csv[train_csv.answered_correctly != -1]

train_csv = train_csv.sort_values(['timestamp'], ascending=True).reset_index(drop = True)

content_mean_final = train_csv[['content_id','answered_correctly']].groupby(['content_id']).agg(['mean'])
content_mean_final.columns = ["answered_correctly_content_mean"]

user_mean_final = train_csv[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean', 'sum', 'count'])
user_mean_final.columns = ["answered_correctly_user_mean", 'sum_correct', 'count']

#saving value to fillna
elapsed_time_mean_final = train_csv.prior_question_elapsed_time.mean()

train_csv.drop(['timestamp', 'content_type_id'], axis=1, inplace=True)


validation = pd.DataFrame()
for i in range(4):
    last_records = train_csv.drop_duplicates('user_id', keep = 'last')
    train_csv = train_csv[~train_csv.index.isin(last_records.index)]
    validation = validation.append(last_records)
X = pd.DataFrame()
for i in range(15):
    last_records = train_csv.drop_duplicates('user_id', keep = 'last')
    train_csv = train_csv[~train_csv.index.isin(last_records.index)]
    X = X.append(last_records)


results_c = train_csv[['content_id','answered_correctly']].groupby(['content_id']).agg(['mean'])
results_c.columns = ["answered_correctly_content_mean"]

results_u = train_csv[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean', 'sum', 'count'])
results_u.columns = ["answered_correctly_user_mean", 'sum_correct', 'count']

result_time_mean = train_csv.prior_question_elapsed_time.mean()

del(train_csv)

X = pd.merge(X, results_u, on=['user_id'], how="left")
X = pd.merge(X, results_c, on=['content_id'], how="left")

validation = pd.merge(validation, results_u, on=['user_id'], how="left")
validation = pd.merge(validation, results_c, on=['content_id'], how="left")

y = X['answered_correctly']
X = X.drop(['answered_correctly'], axis=1)

y_val = validation['answered_correctly']
X_val = validation.drop(['answered_correctly'], axis=1)

lencoder = LabelEncoder()

X['prior_question_had_explanation'].fillna(False, inplace = True)
X['prior_question_had_explanation_enc'] = lencoder.fit_transform(X['prior_question_had_explanation'])
X['answered_correctly_user_mean'].fillna(0.5,  inplace=True)
X['answered_correctly_content_mean'].fillna(0.5,  inplace=True)
X['sum_correct'].fillna(0, inplace = True)
X['count'].fillna(0, inplace = True)
X['prior_question_elapsed_time'].fillna(result_time_mean, inplace = True)

X_val['prior_question_had_explanation'].fillna(False, inplace = True)
X_val['prior_question_had_explanation_enc'] = lencoder.fit_transform(X_val['prior_question_had_explanation'])
X_val['answered_correctly_user_mean'].fillna(0.5,  inplace=True)
X_val['answered_correctly_content_mean'].fillna(0.5,  inplace=True)
X_val['sum_correct'].fillna(0, inplace = True)
X_val['count'].fillna(0, inplace = True)
X_val['prior_question_elapsed_time'].fillna(result_time_mean, inplace = True)

X = X[['answered_correctly_user_mean', 'answered_correctly_content_mean', 'sum_correct', 'count',
       'prior_question_elapsed_time','prior_question_had_explanation_enc']]
X_val = X_val[['answered_correctly_user_mean', 'answered_correctly_content_mean', 'sum_correct', 'count',
       'prior_question_elapsed_time','prior_question_had_explanation_enc']]

scaler = StandardScaler()
X = scaler.fit_transform(X)
X_val = scaler.transform(X_val)


X_train = X.reshape(X.shape[0], 1,X.shape[1])
X_test = X_val.reshape(X_val.shape[0], 1,X_val.shape[1])

In [ ]:
X_train.shape

In [ ]:
class RIIDModel(nn.Module):
  def __init__(self):
    super(RIIDModel,self).__init__()
    self.cnn1 = nn.Conv1d(1,32,2)
    self.cnn2 = nn.Conv1d(32,64,2)
    self.fc1 = nn.Linear(256,32)
    self.fc2 = nn.Linear(32,1)
    self.drop = nn.Dropout(0.1)
    self.relu = nn.ReLU()
  
  def forward(self,x):
    out = self.relu(self.cnn1(x))
    out = self.drop(self.relu(self.cnn2(out)))
    out = out.view(out.shape[0],-1)
    out = self.relu(self.fc1(out))
    out = self.fc2(out)
    return out    

In [ ]:
class RIIDModelLSTM(nn.Module):
  def __init__(self,device):
    super(RIIDModelLSTM,self).__init__()
    self.n_layers = 2
    self.lstm = nn.LSTM(6,6,num_layers = self.n_layers,batch_first=True,dropout=0.3)
    self.fc1 = nn.Linear(6,32)
    self.fc2 = nn.Linear(32,1)
    self.drop = nn.Dropout(0.1)
    self.relu = nn.ReLU()
    self.device = device
  
  def init_hidden(self, batch_size):
    c0 = torch.zeros((self.n_layers, batch_size, 6)).to(self.device)
    h0 = torch.zeros((self.n_layers, batch_size, 6)).to(self.device)
    return h0,c0
  
  def forward(self,x):
    batch_size = x.shape[0]
    h0,c0 = self.init_hidden(batch_size)
    out,_ = self.lstm(x,(h0,c0))
    out = self.drop(self.relu(self.fc1(out)))
    out = self.drop(self.relu(self.fc2(out)))
    return out  

In [ ]:
env = riiideducation.make_env()
iter_test = env.iter_test()

In [ ]:
device = "cpu"
model = RIIDModelLSTM(device)
model.state_dict(torch.load("../input/lstm-model/model2.pt",map_location=device))
model.to(device)
model.eval()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df = pd.merge(test_df, user_mean_final, on=['user_id'],  how="left")
    test_df = pd.merge(test_df, content_mean_final, on=['content_id'],  how="left")
    
    test_df['answered_correctly_user_mean'].fillna(0.6,  inplace=True)
    test_df['answered_correctly_content_mean'].fillna(0.6,  inplace=True)
    test_df['sum_correct'].fillna(0.1, inplace=True)
    test_df['count'].fillna(0.1, inplace=True)
    test_df['prior_question_elapsed_time'].fillna(elapsed_time_mean_final, inplace = True)
    test_df['prior_question_had_explanation'].fillna(False, inplace=True)
    test_df["prior_question_had_explanation_enc"] = lencoder.transform(test_df["prior_question_had_explanation"])

    # fit transform cnn
    X = scaler.transform(test_df[['answered_correctly_user_mean', 'answered_correctly_content_mean', 'sum_correct', 'count',
                                  'prior_question_elapsed_time', 'prior_question_had_explanation_enc']])
    X = X.reshape(X.shape[0], 1, X.shape[1])
    X = torch.tensor(X,dtype=torch.float)
    with torch.no_grad():
        out = model(X)
        out = nn.Sigmoid()(out).view(-1)
    test_df['answered_correctly'] = out.cpu().detach().numpy()
    
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])